# Antaranews dengan textrank

## EDA (Explorasi Data Analysis)  
> proses analisis awal yang dilakukan pada dataset untuk memahami karakteristik, pola, dan struktur data sebelum melakukan analisis lebih lanjut atau membangun model.

### Download dataset

In [ ]:
import os
import pandas as pd
import numpy as np
import gdown
import string

In [ ]:
name = 'data_crawling_berita_antaranews.csv'
gdown.download(f'https://drive.google.com/uc?id=1iuY-raVaRjcwV63Ua8fIGa9dC01yvPvQ', name, quiet=False)

### Baca dataset

In [ ]:
data = pd.read_csv("data_crawling_berita_antaranews.csv")
data

#### Banyak dataset

In [ ]:
len(data)

#### Banyak dataset setiap kelas

In [ ]:
count_ekonomi = 0
count_olahraga= 0
count_politik = 0

for value in data['Label']:
  if value == 'ekonomi':
    count_ekonomi+=1
  elif value == 'olahraga':
    count_olahraga+=1
  else:
    count_politik += 1

print(
f'''
Banyak data ekonomi = {count_ekonomi} data
Banyak data olahraga= {count_olahraga} data
Banyak data politik = {count_politik} data

Total Data          = {len(data)} data
'''
)

## Preprocessing

### Missing Value  
> merupakan nilai pada sebuah data yang kosong/none sehingga harus dihapus untuk proses lebih lanjut

In [ ]:
data.isna().sum()

In [ ]:
data = data.dropna()
data

### Duplicate
> Duplikasi data merujuk pada keadaan di mana ada satu atau lebih salinan dari entri data yang sama atau serupa dalam sebuah dataset. Hal ini berarti terdapat baris atau entri dalam dataset yang memiliki nilai yang identik atau sangat mirip dengan baris lainnya dalam dataset yang sama.

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()
data


In [ ]:
data.reset_index(drop=True, inplace=True) #atur indeks data mulai dari 0 lagi

### **Punctuation process**

> **Punctuation process** merupakan proses normalisasi data yang bertujuan untuk menghilangkan tanda baca



In [ ]:
import re
data['clean_artikel'] = data['Artikel'].str.replace(r'[^\w\s,.?!]', '', regex=True).str.lower()

# Menghilangkan angka dari kolom 'new_abstrak'
data['clean_artikel'] = data['clean_artikel'].str.replace('\d+', '', regex=True)

In [ ]:
data

### **Stopword**

> Stopwords digunakan untuk menghilangkan kata umum yang sering muncul dalam teks seperti: di, dan, atau, dari, ke, saya.



In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')

# Download kamus stop words
nltk.download('stopwords')

In [ ]:
# ditokenizing dulu
data["stopword_artikel"] = data['clean_artikel'].apply(word_tokenize)


# Inisialisasi kamus stop words dari NLTK
stop_words = set(stopwords.words('indonesian'))  # Inisialisasi kamus stop words

# Menghapus stop words dari setiap token
data['stopword_artikel'] = data['stopword_artikel'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])
# Menggabungkan kembali token menjadi kalimat
data['stopword_artikel'] = data['stopword_artikel'].apply(lambda tokens: ' '.join(tokens))
# Membersihkan spasi ganda setelah penghapusan stop words
data['stopword_artikel'] = data['stopword_artikel'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [ ]:
data

### Tokenizing  

> **Tokenizing** adalah proses memecah teks atau dokumen menjadi potongan-potongan yang lebih kecil, yang disebut token. Disini menggunakan term kalimat

In [ ]:
data["tokenizing"] = data['stopword_artikel'].apply(sent_tokenize)

In [ ]:
data

In [ ]:
len(data['tokenizing'][0])

## VSM

### Term Freq

> Term Freq adalah konsep yang digunakan dalam pemrosesan teks dan analisis teks untuk mengukur sejauh mana suatu kata atau term muncul dalam sebuah dokumen atau koleksi dokumen. Term frequency menggambarkan seberapa sering sebuah kata muncul dalam teks relatif terhadap total kata dalam dokumen tersebut.



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
hasil_df_tf = []
set_len_doc = data['tokenizing']

for i in range(len(set_len_doc)): #perulangan setiap artikel/dokumen
  termFreq_vectorizer = CountVectorizer()  #inisialisasi tfidf

  termFreq_matrix = termFreq_vectorizer.fit_transform(data['tokenizing'][i]) #menjadikan tfidf setiap dokumen
  terms = termFreq_vectorizer.get_feature_names_out() #ambil nama fitur

  tf_df = pd.DataFrame(termFreq_matrix.toarray(), columns=terms) #menjadikan dataframe
  hasil_df_tf.append(tf_df) #menyimpan dalam variabel df_tf

In [ ]:
# menampilkan hasil banyak kata pada kalimat

for i in range(len(hasil_df_tf[:10])):
  show_df = hasil_df_tf[i]
  print(f"========= Dokumen ke - {i} ==============")
  display(show_df)
  print("\n")

## Co-occurrence metrics  
> merupakan metrik kemunculan bersama merujuk pada ukuran-ukuran atau metode-metode yang digunakan untuk mengukur sejauh mana dua atau lebih elemen muncul bersama-sama dalam sebuah dataset. Ini dapat diterapkan dalam berbagai konteks, tergantung pada jenis data dan tujuan analisisnya. Berikut adalah beberapa contoh metrik kemunculan bersama yang umum digunakan:

In [ ]:
hasil_co_occurrence_df = [] #untuk menyimpan dataframe co-occurance
hasil_co_occurrence_matrix = [] #menyimpan hasil matrixnya

for i in range(len(hasil_df_tf)):
  # Membuat Co-occurrence Matrix
  co_occurrence_matrix = np.dot(hasil_df_tf[i].T,hasil_df_tf[i])

  # Mengganti diagonal dengan nol (karena kita tidak ingin memperhitungkan kata dengan dirinya sendiri)
  np.fill_diagonal(co_occurrence_matrix, 0)

  # mengambil columns
  kolom = hasil_df_tf[i].columns

  # # Membuat DataFrame Co-occurrence
  co_occurrence_df = pd.DataFrame(co_occurrence_matrix, index=kolom, columns=kolom)

  hasil_co_occurrence_df.append(co_occurrence_df) #simpan dataframe
  hasil_co_occurrence_matrix.append(co_occurrence_matrix) #simpan hasil matrix


In [ ]:
#menampilkan hasil co-occurence

for i in range(len(hasil_co_occurrence_df[:10])):
  print(f"============== Dokumen ke - {i} ==============")
  display(hasil_co_occurrence_df[i])
  print('\n')

## Tambahkan ke graf dan beri treshold

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
graf_results = []
treshold = 0

for i in range(len(hasil_co_occurrence_df)):

  graf = nx.Graph()  # Instantiate as a Graph object

  co_occurrence = hasil_co_occurrence_df[i] #mengambil hasil coocurence per indeks yang disimpan array
  koloms = co_occurrence.columns #mengambil nama kolom saja dari coocurence array indeks
  matrix_coocurrence = hasil_co_occurrence_matrix[i] #mengambil matrix coocurencenya

  for i_koloms in range(len(koloms)): #perulangan setiap dari panjang kolom dari coocurence
      for j_koloms in range(i_koloms + 1, len(koloms)): #perulangan dari setiap panjang kolom + 1, maksudna kolom indeks ke dua
        bobot = matrix_coocurrence[i_koloms,j_koloms] #mencari bobot dengan setiap kolom

        if bobot > treshold: #cek apakah bobot lebih dari treshold
          graf.add_edge(koloms[i_koloms], koloms[j_koloms], weight=matrix_coocurrence[i_koloms, j_koloms])

  graf_results.append(graf)


In [ ]:
# hasil_co_occurrence_df[:1]

In [ ]:
# # Menetapkan treshold
# treshold = 0

# # Inisialisasi list untuk menyimpan hasil graf
# graf_results = []

# # Iterasi melalui setiap DataFrame dalam list
# for i in range(len(hasil_co_occurrence_df)):
#   # Iterasi melalui setiap kolom DataFrame
#   for j in range(len(hasil_co_occurrence_df[i].columns)):
#       # Membuat graf hanya untuk nilai di atas treshold
#       edges_above_threshold = hasil_co_occurrence_df[i][hasil_co_occurrence_df[i][hasil_co_occurrence_df[i].columns[j]] > treshold].index.tolist()
#       G = nx.from_pandas_adjacency(hasil_co_occurrence_df[i].loc[edges_above_threshold, edges_above_threshold])

#       # Menambahkan graf ke list hasil
#       graf_results.append(G)


In [ ]:
len(graf_results)

In [ ]:
# menampilkan graf

for i in range(len(graf_results[:3])):
  pos = nx.spring_layout(graf_results[i])  ## Menentukan posisi/koordinat simpul
  labels = nx.get_edge_attributes(graf_results[i], 'weight') ##mendapatkan atribut berbobot ('weight') dari setiap sisi dalam graf.

  # Menentukan ukuran canvas
  plt.figure(figsize=(50, 30))

  # Menggambar graf dengan ukuran canvas yang diperbesar
  nx.draw(graf_results[i], pos, with_labels=True, node_size=2000, node_color='skyblue')
  nx.draw_networkx_edge_labels(graf_results[i], pos, edge_labels=labels, font_color='red')
  print(f"====== Dokumen ke - {i} ===========")
  plt.show()


## Hitung Centrality

#### Pagerank/Text rank
> TextRank adalah algoritma ekstraksi informasi yang digunakan untuk mengekstrak entitas penting, kata kunci, atau kalimat dari dokumen teks. Algoritma ini didasarkan pada konsep PageRank, yang awalnya dikembangkan untuk mengukur pentingnya halaman web dalam rangkaian halaman web.


##### Langkah-Langkah
1. Preprocessing data dan tokenisasi: Dokumen teks dibagi menjadi token atau kata-kata, dan dilakukan pembersihan teks untuk menghilangkan karakter khusus, tanda baca, dan langkah-langkah preprocessing lainnya.  
2. Vektorisasi: Setiap kata dalam dokumen direpresentasikan sebagai vektor dalam ruang multidimensional. disini menggunakan Term Freq
3. Hitung Simpul simmilarity dan graf
4. Hitung textrank
5. Urutkan

###### Algoritma (Rumus)
> $$
Score(V_i) = (1 - d) + d \times \sum_{j \in In(Vi)} \frac{w_{ji}}{\sum_{k \in Out(V_j)} w_{jk}} \times WScore(V_j)
$$  

> - Score(Vi) adalah skor simpul Vi
> - d adalah faktor damping (dalam TextRank biasanya diatur antara 0.1 hingga 0.3)
> - In Vi adalah himpunan simpul yang memiliki tautan ke Vi
> - Out(Vj) adalah himpunan simpul yang dihubungkan dari Vj
> - Wji adalah bobot dari tautan Vj ke Vi


######  Perhitungan manual  


###### 1.Data

In [ ]:
# set dokumen dulu
# dokumen1 = data['tokenizing'][0][:2]
# dokumen1 = ['Saat ini saya sedang makan','nanti sore saya mengerjakan tugas','setalah tugas tidur']
dokumen1 = ['pagi hari cuaca cerah sekali']

In [ ]:
# df_doc1

In [ ]:
# # preprocessing data
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

# # # Tokenize and filter stopwords
# # dokumen1 = [word for word in word_tokenize(dokumen1[0].lower()) if word.isalnum() and word not in stop_words]

# # print(dokumen1)

# # # Tokenize and filter stopwords for each sentence
# # # Menghapus stop words dari setiap token
# # df_doc1['doc_stopword'] = df_doc1['dokumen'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])
# # # Menggabungkan kembali token menjadi kalimat
# # df_doc1['doc_stopword'] = df_doc1['doc_stopword'].apply(lambda tokens: ' '.join(tokens))
# # # Membersihkan spasi ganda setelah penghapusan stop words
# # df_doc1['doc_stopword'] = df_doc1['doc_stopword'].str.replace(r'\s+', ' ', regex=True).str.strip()


In [ ]:
# dokumen1

###### 2.Vectorizer

In [ ]:
# vectorizer
tf_vectorizer = CountVectorizer()

matrix_dokumen1 = tf_vectorizer.fit_transform(dokumen1)
kata = tf_vectorizer.get_feature_names_out() #ambil nama fitur

df_tf_dokumen1 = pd.DataFrame(matrix_dokumen1.toarray(),columns=kata)

In [ ]:
df_tf_dokumen1

###### 3.Coocurence

In [ ]:
df_cooccurence_dokumen1 = []
matrix_coocurence_dokumen1 = []

coocurence_matrix_dokumen1 = np.dot(df_tf_dokumen1.T, df_tf_dokumen1)
np.fill_diagonal(coocurence_matrix_dokumen1, 0)

koloms = df_tf_dokumen1.columns
df_cooccurence_dokumen1 = pd.DataFrame(coocurence_matrix_dokumen1,index=koloms,columns=koloms)
# coocurence_matrix_dokumen1

In [ ]:
# u = pd.get_dummies(pd.DataFrame(df_tf_dokumen1), prefix='', prefix_sep='').sum(level=0, axis=1)
# v = u.T.dot(u)
# #set 0 to lower triangular matrix
# v.values[np.tril(np.ones(v.shape)).astype(np.bool)] = 0
# print(v)

In [ ]:
df_cooccurence_dokumen1

###### 4.Graff

In [ ]:
len(coocurence_matrix_dokumen1)

In [ ]:
hasil_graf_dokumen1 = nx.DiGraph()
treshold = 0

# Menghitung bobot graph
for i_koloms in range(len(koloms)):
    for j_koloms in range(i_koloms + 1, len(koloms)):
        bobot_dokumen1 = coocurence_matrix_dokumen1[i_koloms, j_koloms]
        if bobot_dokumen1 > treshold:
            print(f'Simpul {koloms[i_koloms]} : {koloms[j_koloms]} = {bobot_dokumen1}')
            hasil_graf_dokumen1.add_edge(koloms[i_koloms], koloms[j_koloms], weight=bobot_dokumen1)


In [ ]:
hasil_graf_dokumen1.edges()

In [ ]:
masuk = dict(hasil_graf_dokumen1.in_degree())
keluar = dict(hasil_graf_dokumen1.out_degree())

pos = nx.spring_layout(hasil_graf_dokumen1)

# Mengakses kunci dari masuk dan keluar dictionaries
node_keys = list(masuk.keys())

# Buat mapping untuk label node
node_labels = {key: f"{koloms[i]} {i + 1}\nIn: {masuk[key]}, Out: {keluar[key]}" for i, key in enumerate(node_keys)}

edge_labels = nx.get_edge_attributes(hasil_graf_dokumen1, 'weight')
# Menggambar graf
plt.figure(figsize=(5, 8))
nx.draw(hasil_graf_dokumen1, pos, with_labels=True, labels=node_labels, font_size=8, font_color='black', node_size=4000, node_color='skyblue', edge_color='gray', linewidths=0.2)
nx.draw_networkx_edge_labels(hasil_graf_dokumen1, pos, edge_labels=edge_labels, font_color='red', font_size=8)

# Menampilkan graf
plt.show()

In [ ]:
# Menghitung text rank
initial_value = 1 / hasil_graf_dokumen1.number_of_nodes()
ranks = {node: initial_value for node in hasil_graf_dokumen1.nodes()}

print(f"Inisialisasi Score Awal : {ranks}")
print(f"Edges : {hasil_graf_dokumen1.edges()}")
print()

damping_factor = 0.85
stopping = 10
tolerance = 1e-4

for i in range(stopping):
    new_ranks = {}
    print(f"========== Iterasi Ke - {i} ==========")

    for node in hasil_graf_dokumen1.nodes():
      rank_sum = 0
      operasi = ''

      # Menampilkan informasi out_sum dan win untuk setiap node dan neighbor
      out_sums_info = {}
      win = hasil_graf_dokumen1.in_degree(node, weight="weight")

      for neighbor in hasil_graf_dokumen1.predecessors(node):
        edge_weight = hasil_graf_dokumen1[neighbor][node]["weight"]
        out_sum = sum(hasil_graf_dokumen1[neighbor][out_neighbor]["weight"] for out_neighbor in hasil_graf_dokumen1.successors(neighbor))
        rank_sum += (edge_weight / out_sum) * ranks[neighbor]
        operasi += f"({edge_weight}/{out_sum}) * {ranks[neighbor]} + "

        # Menambah informasi out_sum untuk setiap neighbor
        out_sums_info[neighbor] = {out_neighbor: hasil_graf_dokumen1[neighbor][out_neighbor]["weight"] for out_neighbor in hasil_graf_dokumen1.successors(neighbor)}

      # Jika operasi kosong, atur nilai operasi menjadi 0
      operasi = operasi[:-3] if operasi else '0'

      new_rank = (1 - damping_factor) + damping_factor * rank_sum
      new_ranks[node] = new_rank

      # Menampilkan informasi out_sum dan win pada setiap iterasi
      print(f"Win({node}): {win}")
      print(f"Out_sums_info({node}): {out_sums_info}")
      print(f"W({node}) = (1 - {damping_factor}) + {damping_factor} * ({operasi}) = {new_rank}")
      print()

    # Periksa konvergensi
    convergence = all(abs(new_ranks[node] - ranks[node]) < tolerance for node in hasil_graf_dokumen1.nodes())
    ranks = new_ranks
    print(f"New Score : {new_ranks}")
    print()

    if convergence:
      print(f"Konvergensi tercapai pada iterasi ke-{i}")
      break

In [ ]:
# ekstrak kata kunci teratas
w = 3
full_text = ' '.join(word for word in dokumen1)

x = dokumen1

# Menghitung nilai dari PageRank (TextRank)
scores = ranks

# Dictionary untuk menyimpan skor tertinggi setiap kata
ranked_words_dict = {}

for word in ' '.join(x).split():
    current_score = scores.get(word, 0)
    if word not in ranked_words_dict or current_score > ranked_words_dict[word]:
        ranked_words_dict[word] = current_score

# Mengurutkan kata-kata berdasarkan skor tertinggi
ranked_words = sorted(((score, word) for word, score in ranked_words_dict.items()), key=lambda x: (x[0], x[1]), reverse=True)

# Memilih sejumlah w kata tertinggi
selected_words = [word for _, word in ranked_words[:w]] if w is not None else None

# Menggabungkan kata-kata menjadi satu string terpisah dengan koma
keywords = ', '.join(selected_words) if selected_words else ''



#### Perbandingan hasil manual dan library  

In [ ]:
# print(f'Dokumen ke {index} : {full_text}')
print(f'{w} Kata Kunci : {keywords}')
print("TextRank Scores:")
for score, word in ranked_words:
    print(f"Skor: {score}, Kata: {word}")

In [ ]:
cen = nx.pagerank(hasil_graf_dokumen1)
cen

In [ ]:
# Mengurutkan dictionary berdasarkan nilai (values) dari yang terbesar
sorted_dict_cen = dict(sorted(cen.items(), key=lambda item: item[1], reverse=True))

# Menampilkan hasil
print(sorted_dict_cen)

In [ ]:
# # Inisialisasi PageRank
# pagerank = {node: 1 / len(hasil_graf_dokumen1.nodes) for node in hasil_graf_dokumen1.nodes}
# # pagerank
# # Iterasi untuk menghitung PageRank
# num_iterations = 3

# for iteration in range(num_iterations):
#     print(f"Iterasi {iteration + 1}:")
#     new_pagerank = {}
#     damping_factor = 0.85

#     for node in hasil_graf_dokumen1.nodes:
#         rank_sum = 0
#         for neighbor in hasil_graf_dokumen1.neighbors(node):
#             print(neighbor)
#             # neighbor_outdegree = keluar[neighbor]
#             # print(neighbor_outdegree)
#             # if neighbor_outdegree == 0:
#             #   neighbor_outdegree = 1
#             # rank_sum += pagerank[neighbor] / neighbor_outdegree

# #         # Hitung PageRank baru menggunakan rumus
# #         new_pagerank[node] = (1 - damping_factor) / len(hasil_graf_dokumen1.nodes) + damping_factor * rank_sum

# #         # Cetak langkah-langkah perhitungan PageRank untuk setiap node
# #         print(f"  - Kalimat {node}: {(1 - damping_factor)}/{len(hasil_graf_dokumen1.nodes)} + ({damping_factor} * {rank_sum}) = {new_pagerank[node]}")


# #     # Perbarui nilai PageRank
# #     pagerank = new_pagerank
# #     print("\n")


In [ ]:
# # Menampilkan hasil PageRank akhir
# print("Hasil Akhir PageRank:")
# for node, rank in pagerank.items():
#     print(f"Kalimat {node}: {rank}")

In [ ]:
# # inisilaisasi dumping factor / d

# d = nx.pagerank(graf_results[i])

### Ranking kata kunci All doc

In [ ]:
centrality_result = []

for i in range(len(graf_results)):

  centrality = nx.pagerank(graf_results[i]) #menjadikan graf setiap index list diclosness

  centrality_result.append(centrality) #menyimpan hasil centrality


In [ ]:
# menampilkan centrality
for i in range(len(centrality_result[:5])):
  print(f"================== Dokumen ke - {i} ==================")
  print(f"Text rank score : {centrality_result[i]}")

In [ ]:
# # mengurutkan hasil closeness
# sorted_closeness_all = []
# for i in range(len(centrality_result)):
#   sorted_closeness = dict(sorted(centrality_result[i].items(), key=lambda item: item[1], reverse=True))
#   sorted_closeness_all.append(sorted_closeness)


In [ ]:
# List untuk menyimpan hasil
top_5_per_document = []

# Mengambil 5 data tertinggi untuk setiap dictionary
for doc in centrality_result:
    # Mengurutkan dictionary berdasarkan nilai (values)
    sorted_doc = dict(sorted(doc.items(), key=lambda item: item[1], reverse=True))

    # Mengambil 5 data tertinggi
    top_5_data = dict(list(sorted_doc.items())[:5])

    # Menambahkan ke list hasil
    top_5_per_document.append(top_5_data)



In [ ]:
top_5_per_document

In [ ]:
len(top_5_per_document)

In [ ]:
top_5_per_document[0]

In [ ]:

for i in range(len(top_5_per_document)):
  print(f"==== Dokumen ke-{i} ====")
  for key,value in top_5_per_document[i].items():
    print(f'Kata kunci : {key} => {value}')
  print('\n')